In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import os
import tqdm

Using TensorFlow backend.


In [76]:
auggen = ImageDataGenerator(rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=True)

imread('data/raw/train/walleye/005597a253936bb45edf70392d89b403ee7d2285.jpg')
next(auggen.flow(np.tile(np.empty((1, 224, 224, 3)), (4, 1, 1, 1)),
            np.empty((4, 4)), batch_size=4))[0].shape

(4, 224, 224, 3)

In [73]:
np.tile(imread('data/raw/train/walleye/005597a253936bb45edf70392d89b403ee7d2285.jpg'), (4, 1, 1, 1)).shape


(4, 256, 256, 3)

In [44]:
import os
import subprocess

import tqdm

import numpy as np

from keras.preprocessing.image import load_img


def imgdir_to_arr(data_dir, arr_dir):
    print('Converting from: "{}"'.format(data_dir))
    print('Saving to: "{}"'.format(arr_dir))

    subprocess.call(['mkdir', '-p', arr_dir])

    cats = sorted(os.listdir(data_dir))
    cat_nbr = len(cats)
    print('Iterating over all categories')
    
    for cat_idx, cat in tqdm.tqdm(enumerate(cats)):
        cat_path = os.path.join(data_dir, cat)
        img_files = sorted(os.listdir(cat_path))
        for img_idx, img_file in enumerate(img_files):
            img_path = os.path.join(cat_path, img_file)
            img = (img_path)
            img_name = '{:04d}-img-{}-{}'.format(img_idx, cat, cat_idx)
            lab_name = '{:04d}-lab-{}-{}'.format(img_idx, cat, cat_idx)
            lab = np.eye(cat_nbr, dtype=np.float32)[cat_idx]
            arr_path = os.path.join(arr_dir, img_name)
            lab_path = os.path.join(arr_dir, lab_name)
            np.save(arr_path, img)
            np.save(lab_path, lab)


if __name__ == '__main__':
    imgdir_to_arr('data/raw/train', 'data/arr/train')
    imgdir_to_arr('data/raw/test', 'data/arr/test')

0it [00:00, ?it/s]

Converting from: "data/raw/train"
Saving to: "data/arr/train"
Iterating over all categories


4it [00:03,  1.12it/s]
0it [00:00, ?it/s]

Converting from: "data/raw/test"
Saving to: "data/arr/test"
Iterating over all categories


4it [00:01,  2.82it/s]


In [40]:
arr_dir='data/arr/test'

def gen_XY_from_dir(arr_dir):
    arr_files = sorted(os.listdir(arr_dir))
    arr_names = list(filter(lambda x: r'-img-' in x, arr_files))
    lab_names = list(filter(lambda x: r'-lab-' in x, arr_files))

    assert len(arr_names) == len(lab_names), '# labels != images'

    for arr_name, lab_name in zip(arr_names, lab_names):
        X = np.load(os.path.join(arr_dir, arr_name))
        Y = np.load(os.path.join(arr_dir, lab_name))
        yield X, Y

next(gen_XY_from_dir('data/arr/test'))

(array([[[215, 200, 167],
         [204, 189, 156],
         [229, 214, 181],
         ..., 
         [147, 156, 173],
         [121, 128, 144],
         [181, 188, 204]],
 
        [[210, 197, 163],
         [211, 198, 164],
         [224, 211, 177],
         ..., 
         [144, 153, 170],
         [126, 135, 152],
         [175, 184, 201]],
 
        [[202, 189, 155],
         [207, 194, 160],
         [208, 195, 161],
         ..., 
         [133, 144, 162],
         [129, 140, 158],
         [162, 174, 190]],
 
        ..., 
        [[139, 128,  96],
         [143, 132, 102],
         [144, 133, 103],
         ..., 
         [168, 159, 116],
         [155, 146, 103],
         [147, 139,  93]],
 
        [[136, 127,  94],
         [141, 132, 101],
         [143, 132, 102],
         ..., 
         [156, 148, 101],
         [146, 138,  91],
         [146, 138,  91]],
 
        [[135, 126,  93],
         [140, 131, 100],
         [142, 133, 102],
         ..., 
         [151, 140,  94

In [6]:
import os
import subprocess

import numpy as np

from keras.preprocessing.image import ImageDataGenerator


aug_rounds = 4


# TODO: replace this listdir with a mapping tbl/json
CATS = os.listdir('data/raw/train')
def cat_from_int(cat_int):
    return CATS[cat_int]


def gen_XY_from_dir(arr_dir):
    arr_files = sorted(os.listdir(arr_dir))
    arr_names = list(filter(lambda x: r'-img-' in x, arr_files))
    lab_names = list(filter(lambda x: r'-lab-' in x, arr_files))

    assert len(arr_names) == len(lab_names), '# labels != images'

    for arr_name, lab_name in zip(arr_names, lab_names):
        X = np.load(os.path.join(arr_dir, arr_name))
        Y = np.load(os.path.join(arr_dir, lab_name))
        yield X, Y


def augment_XY(x, y, aug_rounds):
    auggen = ImageDataGenerator(rotation_range=10,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                horizontal_flip=True)

    X_aug, Y_aug = next(auggen.flow([x], [y],
                                    target_size=(img_width, img_height),
                                    batch_size=aug_rounds))

    for x_aug, y_aug in zip(X_aug, Y_aug):
        yield x_aug, y_aug


def arrs_to_aug(arr_dir, aug_dir):
    
    subprocess.call(['mkdir', '-p', aug_dir])

    for img_idx, (x, y) in enumerate(gen_XY_from_dir(arr_dir)):
        for aug_idx, (x_aug, y_aug) in augment_XY(x, y, aug_rounds):
            cat_idx = np.argmax(y_aug)
            cat = cat_from_int(cat_idx)
            img_name = '{:04d}-{:02d}-img-{}-{}'.format(img_idx, aug_idx cat, cat_idx)
            lab_name = '{:04d}-{:02d}-lab-{}-{}'.format(img_idx, aug_idx cat, cat_idx)
            arr_path = os.path.join(arr_dir, img_name)
            lab_path = os.path.join(arr_dir, lab_name)
            np.save(aug_path, x_aug)
            np.save(aug_path, y_aug)


if __name__ == '__main__':
    arrs_to_aug('data/train/arr', 'data/train/aug')
    arrs_to_aug('data/test/arr', 'data/test/aug')




0it [00:00, ?it/s]

Found 1244 images belonging to 4 classes.




1it [00:00,  4.79it/s]

2it [00:00,  4.87it/s]

3it [00:00,  4.93it/s]

4it [00:00,  4.98it/s]

5it [00:01,  4.96it/s]

6it [00:01,  4.88it/s]

7it [00:01,  4.67it/s]

8it [00:01,  4.59it/s]

9it [00:01,  4.74it/s]

10it [00:02,  4.82it/s]

11it [00:02,  4.82it/s]

12it [00:02,  4.88it/s]

13it [00:02,  4.93it/s]

14it [00:02,  4.96it/s]

15it [00:03,  4.99it/s]

16it [00:03,  4.51it/s]

17it [00:03,  4.52it/s]

18it [00:03,  4.30it/s]

19it [00:04,  4.31it/s]

20it [00:04,  4.52it/s]

21it [00:04,  4.68it/s]

22it [00:04,  4.80it/s]

23it [00:04,  4.87it/s]

24it [00:05,  4.53it/s]

25it [00:05,  4.11it/s]

26it [00:05,  4.07it/s]

27it [00:05,  4.32it/s]

28it [00:06,  4.54it/s]

29it [00:06,  4.70it/s]

30it [00:06,  4.81it/s]

31it [00:06,  4.88it/s]

32it [00:06,  4.96it/s]

33it [00:07,  4.98it/s]

34it [00:07,  4.59it/s]

35it [00:07,  4.40it/s]

36it [00:07,  4.57it/s]

37it [00:07,  4.55it/s]

38it [00:08,  4.15it/s]

39it [00:08,  3.99it/s]

40it [00:08,  4.24it/s]

41it [0

KeyboardInterrupt: 

In [51]:
list(zip(np.array([[1,1],[1,1]]), np.array([[2,2],[2,2]])))

[(array([1, 1]), array([2, 2])), (array([1, 1]), array([2, 2]))]

In [3]:
data_dir = 'data/train'
batch_count = 50
img_width = 224
img_height = 224
batch_size = 16

def transform_batches(data_dir, data_gen_dir):
    X_transformed = []
    Y = []
    datagen = ImageDataGenerator(
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True)

    count = 0
    for X_batch, Y_batch in datagen.flow_from_directory(
            data_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode='binary',
            shuffle=True,
            save_to_dir=data_gen_dir):
        print('Batch ', count, '/', batch_count)
        X_transformed.append(model.predict_on_batch(X_batch))
        Y.append(Y_batch)
        count += 1
        if len(Y) == batch_count:
            print('All done, gubna!')
            X_transformed = np.concatenate(X_transformed)
            Y = np.concatenate(Y)
            break
    
    return X_transformed, Y

def train_top_model(X_train, Y_train, X_test, Y_test, top_model_weights_path, epochs = 50, batch_size = 32):
    
    model = Sequential()
    model.add(Flatten(input_shape=X_train.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, Y_train,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(X_test, Y_test))
    model.save_weights(top_model_weights_path)
    return model

In [4]:
model = applications.ResNet50(include_top=False, weights='imagenet')

X_test, Y_test = transform_batches('data/test', 'data/generated')

X_train, Y_train = transform_batches('data/train', 'data/generated')

np.save('data/X_test_transformed.npy',X_test)
np.save('data/Y_test_transformed.npy',Y_test)
np.save('data/X_train_transformed.npy',X_train)
np.save('data/Y_train_transformed.npy',Y_train)

Found 270 images belonging to 2 classes.
Batch  0 / 50
Batch  1 / 50
Batch  2 / 50
Batch  3 / 50
Batch  4 / 50
Batch  5 / 50
Batch  6 / 50
Batch  7 / 50
Batch  8 / 50
Batch  9 / 50
Batch  10 / 50
Batch  11 / 50
Batch  12 / 50
Batch  13 / 50
Batch  14 / 50
Batch  15 / 50
Batch  16 / 50
Batch  17 / 50
Batch  18 / 50
Batch  19 / 50
Batch  20 / 50
Batch  21 / 50
Batch  22 / 50
Batch  23 / 50
Batch  24 / 50
Batch  25 / 50
Batch  26 / 50
Batch  27 / 50
Batch  28 / 50
Batch  29 / 50
Batch  30 / 50
Batch  31 / 50
Batch  32 / 50
Batch  33 / 50
Batch  34 / 50
Batch  35 / 50
Batch  36 / 50
Batch  37 / 50
Batch  38 / 50
Batch  39 / 50
Batch  40 / 50
Batch  41 / 50
Batch  42 / 50
Batch  43 / 50
Batch  44 / 50
Batch  45 / 50
Batch  46 / 50
Batch  47 / 50
Batch  48 / 50
Batch  49 / 50
All done, gubna!
Found 627 images belonging to 2 classes.
Batch  0 / 50
Batch  1 / 50
Batch  2 / 50
Batch  3 / 50
Batch  4 / 50
Batch  5 / 50
Batch  6 / 50
Batch  7 / 50
Batch  8 / 50
Batch  9 / 50
Batch  10 / 50
Batch 

In [5]:
top_model = train_top_model(X_train, Y_train, X_test, Y_test, 'data/resnet_1.h5')

Train on 787 samples, validate on 796 samples
Epoch 1/50
787/787 [==============================] - 0s - loss: 0.9830 - acc: 0.7243 - val_loss: 0.2685 - val_acc: 0.9183
Epoch 2/50
787/787 [==============================] - 0s - loss: 0.3860 - acc: 0.8424 - val_loss: 0.2080 - val_acc: 0.9171
Epoch 3/50
787/787 [==============================] - 0s - loss: 0.3461 - acc: 0.8793 - val_loss: 0.2110 - val_acc: 0.9095
Epoch 4/50
787/787 [==============================] - 0s - loss: 0.2050 - acc: 0.9263 - val_loss: 0.2127 - val_acc: 0.9045
Epoch 5/50
787/787 [==============================] - 0s - loss: 0.1938 - acc: 0.9225 - val_loss: 0.3896 - val_acc: 0.8430
Epoch 6/50
787/787 [==============================] - 0s - loss: 0.1905 - acc: 0.9238 - val_loss: 0.1865 - val_acc: 0.9108
Epoch 7/50
787/787 [==============================] - 0s - loss: 0.1495 - acc: 0.9377 - val_loss: 0.4041 - val_acc: 0.8518
Epoch 8/50
787/787 [==============================] - 0s - loss: 0.1515 - acc: 0.9441 - val_l

In [6]:
def train_top_model_2(X_train, Y_train, X_test, Y_test, top_model_weights_path, epochs = 50, batch_size = 32):
    
    model = Sequential()
    model.add(Flatten(input_shape=X_train.shape[1:]))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, Y_train,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(X_test, Y_test))
    model.save_weights(top_model_weights_path)
    return model

In [8]:
top_model = train_top_model_2(X_train, Y_train, X_test, Y_test, 'data/resnet_2.h5')

Train on 787 samples, validate on 796 samples
Epoch 1/50
787/787 [==============================] - 2s - loss: 5.3176 - acc: 0.6315 - val_loss: 5.4076 - val_acc: 0.6608
Epoch 2/50
787/787 [==============================] - 2s - loss: 3.7993 - acc: 0.6099 - val_loss: 0.7194 - val_acc: 0.5013
Epoch 3/50
787/787 [==============================] - 1s - loss: 0.6630 - acc: 0.7484 - val_loss: 0.5598 - val_acc: 0.7412
Epoch 4/50
787/787 [==============================] - 2s - loss: 0.4385 - acc: 0.8386 - val_loss: 0.2245 - val_acc: 0.9033
Epoch 5/50
787/787 [==============================] - 1s - loss: 0.3025 - acc: 0.8869 - val_loss: 0.2518 - val_acc: 0.8920
Epoch 6/50
787/787 [==============================] - 1s - loss: 0.3301 - acc: 0.8793 - val_loss: 0.1999 - val_acc: 0.9095
Epoch 7/50
787/787 [==============================] - 1s - loss: 0.2711 - acc: 0.9047 - val_loss: 0.2973 - val_acc: 0.8568
Epoch 8/50
787/787 [==============================] - 2s - loss: 0.2321 - acc: 0.9212 - val_l

In [9]:
# added white and yellow perch from @MattGraz

In [12]:

batch_count = 100
X_test, Y_test = transform_batches('data/test', None)

X_train, Y_train = transform_batches('data/train', None)

np.save('data/X_test_transformed_4_fish.npy',X_test)
np.save('data/Y_test_transformed_4_fish.npy',Y_test)
np.save('data/X_train_transformed_4_fish.npy',X_train)
np.save('data/Y_train_transformed_4_fish.npy',Y_train)

Found 535 images belonging to 4 classes.
Batch  0 / 100
Batch  1 / 100
Batch  2 / 100
Batch  3 / 100
Batch  4 / 100
Batch  5 / 100
Batch  6 / 100
Batch  7 / 100
Batch  8 / 100
Batch  9 / 100
Batch  10 / 100
Batch  11 / 100
Batch  12 / 100
Batch  13 / 100
Batch  14 / 100
Batch  15 / 100
Batch  16 / 100
Batch  17 / 100
Batch  18 / 100
Batch  19 / 100
Batch  20 / 100
Batch  21 / 100
Batch  22 / 100
Batch  23 / 100
Batch  24 / 100
Batch  25 / 100
Batch  26 / 100
Batch  27 / 100
Batch  28 / 100
Batch  29 / 100
Batch  30 / 100
Batch  31 / 100
Batch  32 / 100
Batch  33 / 100
Batch  34 / 100
Batch  35 / 100
Batch  36 / 100
Batch  37 / 100
Batch  38 / 100
Batch  39 / 100
Batch  40 / 100
Batch  41 / 100
Batch  42 / 100
Batch  43 / 100
Batch  44 / 100
Batch  45 / 100
Batch  46 / 100
Batch  47 / 100
Batch  48 / 100
Batch  49 / 100
Batch  50 / 100
Batch  51 / 100
Batch  52 / 100
Batch  53 / 100
Batch  54 / 100
Batch  55 / 100
Batch  56 / 100
Batch  57 / 100
Batch  58 / 100
Batch  59 / 100
Batch  60

In [61]:
def train_top_model_3(X_train, Y_train, X_test, Y_test, top_model_weights_path, epochs = 50, batch_size = 32):
    
    model = Sequential()
    model.add(Flatten(input_shape=X_train.shape[1:]))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(X_train, Y_train,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(X_test, Y_test))
    model.save_weights(top_model_weights_path)
    return model

In [58]:
from sklearn.preprocessing import OneHotEncoder

In [59]:
ohe = OneHotEncoder()
Y_train_transform = ohe.fit_transform(Y_train.reshape(-1,1))

Y_test_transform = ohe.transform(Y_test.reshape(-1,1))

In [62]:
model_3 = train_top_model_3(X_train, Y_train_transform.todense(), X_test, Y_test_transform.todense(), 'data/resnet_3.h5', epochs=1000)

Train on 1596 samples, validate on 1582 samples
Epoch 1/1000
1596/1596 [==============================] - 5s - loss: 2.0263 - acc: 0.2769 - val_loss: 1.3586 - val_acc: 0.3344
Epoch 2/1000
1596/1596 [==============================] - 5s - loss: 1.4039 - acc: 0.3227 - val_loss: 1.3059 - val_acc: 0.3831
Epoch 3/1000
1596/1596 [==============================] - 4s - loss: 1.2590 - acc: 0.4160 - val_loss: 1.1440 - val_acc: 0.5790
Epoch 4/1000
1596/1596 [==============================] - 4s - loss: 1.0918 - acc: 0.5414 - val_loss: 0.9827 - val_acc: 0.6334
Epoch 5/1000
1596/1596 [==============================] - 4s - loss: 0.9679 - acc: 0.6034 - val_loss: 0.9226 - val_acc: 0.6593
Epoch 6/1000
1596/1596 [==============================] - 5s - loss: 0.8578 - acc: 0.6566 - val_loss: 0.7932 - val_acc: 0.7067
Epoch 7/1000
1596/1596 [==============================] - 4s - loss: 0.7892 - acc: 0.6911 - val_loss: 0.7764 - val_acc: 0.7257
Epoch 8/1000
1596/1596 [==============================] - 4s - 

In [63]:

batch_count = 50
model = applications.ResNet50(include_top=False, weights='imagenet')

X_test, Y_test = transform_batches('data/test2', 'data/generated-perch')

X_train, Y_train = transform_batches('data/train2', 'data/generated-perch')

np.save('data/X_test_transformed_perch.npy',X_test)
np.save('data/Y_test_transformed_perch.npy',Y_test)
np.save('data/X_train_transformed_perch.npy',X_train)
np.save('data/Y_train_transformed_perch.npy',Y_train)

Found 265 images belonging to 2 classes.
Batch  0 / 50
Batch  1 / 50
Batch  2 / 50
Batch  3 / 50
Batch  4 / 50
Batch  5 / 50
Batch  6 / 50
Batch  7 / 50
Batch  8 / 50
Batch  9 / 50
Batch  10 / 50
Batch  11 / 50
Batch  12 / 50
Batch  13 / 50
Batch  14 / 50
Batch  15 / 50
Batch  16 / 50
Batch  17 / 50
Batch  18 / 50
Batch  19 / 50
Batch  20 / 50
Batch  21 / 50
Batch  22 / 50
Batch  23 / 50
Batch  24 / 50
Batch  25 / 50
Batch  26 / 50
Batch  27 / 50
Batch  28 / 50
Batch  29 / 50
Batch  30 / 50
Batch  31 / 50
Batch  32 / 50
Batch  33 / 50
Batch  34 / 50
Batch  35 / 50
Batch  36 / 50
Batch  37 / 50
Batch  38 / 50
Batch  39 / 50
Batch  40 / 50
Batch  41 / 50
Batch  42 / 50
Batch  43 / 50
Batch  44 / 50
Batch  45 / 50
Batch  46 / 50
Batch  47 / 50
Batch  48 / 50
Batch  49 / 50
All done, gubna!
Found 617 images belonging to 2 classes.
Batch  0 / 50
Batch  1 / 50
Batch  2 / 50
Batch  3 / 50
Batch  4 / 50
Batch  5 / 50
Batch  6 / 50
Batch  7 / 50
Batch  8 / 50
Batch  9 / 50
Batch  10 / 50
Batch 

In [64]:
perch_model = train_top_model_2(X_train, Y_train, X_test, Y_test, 'data/resnet_2_perch.h5')

Train on 793 samples, validate on 786 samples
Epoch 1/50
793/793 [==============================] - 2s - loss: 2.4071 - acc: 0.5259 - val_loss: 0.6271 - val_acc: 0.6527
Epoch 2/50
793/793 [==============================] - 1s - loss: 0.7912 - acc: 0.6381 - val_loss: 0.5839 - val_acc: 0.7074
Epoch 3/50
793/793 [==============================] - 1s - loss: 0.7074 - acc: 0.6923 - val_loss: 0.4824 - val_acc: 0.7901
Epoch 4/50
793/793 [==============================] - 1s - loss: 0.5565 - acc: 0.7427 - val_loss: 0.4897 - val_acc: 0.7316
Epoch 5/50
793/793 [==============================] - 1s - loss: 0.5325 - acc: 0.7768 - val_loss: 0.5720 - val_acc: 0.7366
Epoch 6/50
793/793 [==============================] - 1s - loss: 0.4697 - acc: 0.8159 - val_loss: 0.5298 - val_acc: 0.7634
Epoch 7/50
793/793 [==============================] - 1s - loss: 0.4191 - acc: 0.8058 - val_loss: 0.5962 - val_acc: 0.7710
Epoch 8/50
793/793 [==============================] - 1s - loss: 0.4022 - acc: 0.8487 - val_l

In [72]:
def train_top_model_5(X_train, Y_train, X_test, Y_test, top_model_weights_path, epochs = 50, batch_size = 32):
    
    model = Sequential()
    model.add(Flatten(input_shape=X_train.shape[1:]))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, Y_train,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(X_test, Y_test))
    model.save_weights(top_model_weights_path)
    return model

In [73]:
perch_model = train_top_model_5(X_train, Y_train, X_test, Y_test, 'data/resnet_5_perch.h5', epochs=500)

Train on 793 samples, validate on 786 samples
Epoch 1/500
793/793 [==============================] - 3s - loss: 1.3682 - acc: 0.5057 - val_loss: 0.6319 - val_acc: 0.6056
Epoch 2/500
793/793 [==============================] - 2s - loss: 0.9147 - acc: 0.5574 - val_loss: 0.5927 - val_acc: 0.7214
Epoch 3/500
793/793 [==============================] - 2s - loss: 0.7189 - acc: 0.6141 - val_loss: 0.5617 - val_acc: 0.7506
Epoch 4/500
793/793 [==============================] - 2s - loss: 0.5432 - acc: 0.7352 - val_loss: 0.5026 - val_acc: 0.7774
Epoch 5/500
793/793 [==============================] - 2s - loss: 0.4371 - acc: 0.8096 - val_loss: 0.4606 - val_acc: 0.7939
Epoch 6/500
793/793 [==============================] - 2s - loss: 0.3956 - acc: 0.8310 - val_loss: 0.4605 - val_acc: 0.7824
Epoch 7/500
793/793 [==============================] - 2s - loss: 0.3378 - acc: 0.8638 - val_loss: 0.4755 - val_acc: 0.7888
Epoch 8/500
793/793 [==============================] - 2s - loss: 0.3105 - acc: 0.8625